》train_set.csv：此数据集用于训练模型，每一行对应一篇文章。文章分别在“字”和“词”的级别上做了脱敏处理。共有四列：
第一列是文章的索引(id)，第二列是文章正文在“字”级别上的表示，即字符相隔正文(article)；第三列是在“词”级别上的表示，即词语相隔正文(word_seg)；第四列是这篇文章的标注(class)。
注：每一个数字对应一个“字”，或“词”，或“标点符号”。“字”的编号与“词”的编号是独立的！

》test_set.csv：此数据用于测试。数据格式同train_set.csv，但不包含class。
注：test_set与train_test中文章id的编号是独立的。

In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time


In [2]:

train = pd.read_csv('C:/daguan_text_classification/new_data/train_set.csv')
test = pd.read_csv('C:/daguan_text_classification/new_data/test_set.csv')
test_id = pd.read_csv('C:/daguan_text_classification/new_data/test_set.csv')[["id"]].copy()



In [3]:
train.head()

,id,article,word_seg,class
0,0,7368 1252069 365865 755561 1044285 129532 1053...,816903 597526 520477 1179558 1033823 758724 63...,14
1,1,581131 165432 7368 957317 1197553 570900 33659...,90540 816903 441039 816903 569138 816903 10343...,3
2,2,7368 87936 40494 490286 856005 641588 145611 1...,816903 1012629 957974 1033823 328210 947200 65...,12
3,3,299237 760651 299237 887082 159592 556634 7489...,563568 1239563 680125 780219 782805 1033823 19...,13
4,4,7368 7368 7368 865510 7368 396966 995243 37685...,816903 816903 816903 139132 816903 312320 1103...,12


In [4]:
test.head()

,id,article,word_seg
0,0,7368 146447 316564 42610 55736 297797 93042 53...,816903 565958 726082 764656 335008 75094 20282...
1,1,985531 473628 1044285 1121849 206763 462208 11...,729468 520477 529032 101368 335130 520477 1113...
2,2,7368 7368 7368 7368 7368 7368 7368 7368 7368 7...,816903 816903 816903 816903 816903 816903 8169...
3,3,529819 1226459 856005 1177293 663773 272235 93...,231664 1033823 524850 330478 507199 520477 618...
4,4,42610 1252069 1077049 955883 1125260 1044285 2...,545370 379223 162767 520477 1194630 1197475 11...


In [5]:
train.shape

(102277, 4)

In [6]:
test.shape

(102277, 3)

In [7]:
# import seaborn as sns
# sns.countplot(train['class'])
# print(' percentage:',len(np.argwhere(train['class'] == 5))/len(train))

## Baseline1 -  TF-IDF + chi-square + SVM 0.77854

In [8]:
t0 = time.time()
import pandas as pd, numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import svm
from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif

column = "word_seg"
train = pd.read_csv('C:/daguan_text_classification/new_data/train_set.csv')
test = pd.read_csv('C:/daguan_text_classification/new_data/test_set.csv')
test_id = test["id"].copy()
vec = TfidfVectorizer(ngram_range=(1,2),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
trn_term_doc = vec.fit_transform(train[column])
test_term_doc = vec.transform(test[column])



In [10]:

# from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
# fselect = SelectKBest(chi2 , k=2000000)
# train_features = fselect.fit_transform(trn_term_doc, train["class"])
# test_features = fselect.transform(test_term_doc)


fid0=open('baseline.csv','w')

y=(train["class"]-1).astype(int)
lin_clf = svm.LinearSVC(C=0.95,verbose=4)
lin_clf.fit(trn_term_doc,y)
preds = lin_clf.predict(test_term_doc)
i=0
fid0.write("id,class"+"\n")
for item in preds:
    fid0.write(str(i)+","+str(item+1)+"\n")
    i=i+1
fid0.close()
t1 = time.time()
print(t1-t0)

[LibLinear]804.3816695213318


BaseLine2. 

In [11]:
column = "word_seg"
vec = CountVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.98, binary=True)
vec.fit(train[column])
train_feature_names = vec.get_feature_names()
del vec


vec1 = CountVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.98,  binary=True)
vec1.fit(test[column])
test_features_names = vec1.get_feature_names()
del vec1

In [12]:
valid_feature_name = list(set(train_feature_names+test_features_names))
len(valid_feature_name)

3702540

In [13]:
join_df = train[['word_seg','class']].groupby('class').agg(lambda ele:' '.join(ele))

区别于直接拿每一个文档计算df，把每一个groupby之后再去算tf-i(class)f

In [22]:
vec2 = TfidfVectorizer(ngram_range=(1, 2), use_idf=1, smooth_idf=1, 
                           sublinear_tf=1,vocabulary=valid_feature_name)
vec2.fit(join_df[column])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words=None, strip_accents=None, sublinear_tf=1,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=1,
        vocabulary=['455167 819406', '411707 1266986', '473339', '739365 497479', '1171578 1063907', '47305 652252', '1057288 526298', '647324 374759', '516889 439312', '943146 685057', '329046', '201133 862083', '778749 1216135', '1134165 1205782', '99575 99203', '670748 432126', '520477 546009', '809608 9...909613 1033823', '598948 239905', '680125 587064', '133940 975921', '44243 995362', '957933 308610'])

In [24]:
train_X = vec2.transform(train[column])
train_Y = train['class']
test_X = vec2.transform(test[column])


In [26]:
from sklearn import svm
clf = svm.LinearSVC(tol=1, class_weight='balanced')
clf.fit(train_X,train_Y)
pred = clf.predict(test_X)


In [28]:
out_df = pd.DataFrame(index=test.index)
out_df['class']=pred
out_df.to_csv('baseline2.csv',index=True, header=True)